In [ ]:
import os
import time
import csv
import re # 필요한 경우 텍스트 정제에 사용될 수 있으므로 유지합니다.
from dotenv import load_dotenv
import random # time.sleep()에 사용

# 웹 스크레이핑 관련 라이브러리는 더 이상 필요하지 않습니다.
# import requests
# from bs4 import BeautifulSoup
from tqdm import tqdm

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.pydantic_v1 import BaseModel, Field

# --- 1. 설정 ---
load_dotenv()
API_KEY = os.getenv('GEMINI_API') # .env 파일의 변수명을 GEMINI_API_KEY로 가정합니다 (일반적). 필요시 GEMINI_API로 수정하세요.

if not API_KEY:
    print("오류: GEMINI_API_KEY 환경 변수를 설정해주세요.")
    exit()

LANGCHAIN_MODEL_NAME = 'gemini-2.0-flash' # 사용 가능한 모델명으로 수정했습니다. 필요시 gemini-2.0-flash 등으로 변경하세요.

try:
    llm = ChatGoogleGenerativeAI(
        google_api_key=API_KEY,
        model=LANGCHAIN_MODEL_NAME,
        temperature=0.3, # 욕설 판단은 창의성보다 일관성이 중요하므로 온도를 낮춥니다.
        top_p=1,
        top_k=1,
    )
except Exception as e:
    print(f"LangChain LLM ({LANGCHAIN_MODEL_NAME}) 초기화 중 오류 발생: {e}")
    print("GEMINI_API_KEY가 정확하고, langchain_google_genai 라이브러리가 올바르게 설치되었는지, 모델명이 유효한지 확인해주세요.")
    exit()

INPUT_CSV_FILE = "./data/raw_data.csv" # 입력 CSV 파일 경로
OUTPUT_CSV_FILE = "./data/1st_filtering.csv" # 출력 CSV 파일명 변경
# HEADERS는 웹 스크레이핑에 사용되므로 더 이상 필요하지 않습니다.

# --- 2. CSV 데이터 로드 함수 ---
def load_data_from_csv(file_path):
    """CSV 파일에서 데이터를 읽어옵니다."""
    data = []
    try:
        with open(file_path, mode='r', encoding='utf-8-sig') as file:
            reader = csv.DictReader(file)
            if not reader.fieldnames or not all(col in reader.fieldnames for col in ['title', 'link', 'content_excerpt']):
                print(f"오류: CSV 파일 '{file_path}'에 'title', 'link', 'content_excerpt' 열이 모두 존재해야 합니다.")
                return []
            for row in reader:
                # 누락된 열이 있는지 확인하고, 있다면 기본값으로 채우거나 오류 처리
                title = row.get('title', '제목 없음')
                link = row.get('link', '링크 없음')
                content = row.get('content_excerpt', '') # 분석할 내용
                if not content: # content_excerpt가 비어있는 경우 분석에서 제외하거나 특별 처리 가능
                    print(f"경고: '{title}' 게시글의 'content_excerpt'가 비어있습니다. 분석에서 제외될 수 있습니다.")
                data.append({'title': title, 'link': link, 'content': content})
        print(f"'{file_path}'에서 총 {len(data)}개의 데이터를 로드했습니다.")
    except FileNotFoundError:
        print(f"오류: 입력 CSV 파일 '{file_path}'을(를) 찾을 수 없습니다.")
        return []
    except Exception as e:
        print(f"오류: CSV 파일 '{file_path}' 로드 중 문제 발생 - {e}")
        return []
    return data

# --- 3. LangChain을 사용한 텍스트 분석 함수 ---

# Pydantic 모델 정의 (LLM이 반환할 JSON 구조 명시)
class ProfanityAnalysis(BaseModel):
    classification: str = Field(description="게시글 내용에 직접적인 욕설이 포함되어 있으면 '욕설 사용', 아니면 '일반글'")
    reason: str = Field(description="분석 결과에 대한 이유. '욕설 사용' 시 '직접적인 욕설 포함됨'. '일반글' 시 '해당 없음' 또는 '직접적인 욕설 없음'")
    confidence_score: float = Field(description="분석 결과에 대한 신뢰도 점수 (0.0 ~ 1.0), 소수점 둘째자리까지")

def analyze_text_with_langchain(text_content, post_title, post_link):
    """LangChain과 ChatGoogleGenerativeAI를 사용하여 텍스트에 직접적인 욕설이 있는지 분석합니다."""
    if not text_content or not text_content.strip():
        return {"classification": "분석 불가", "reason": "분석할 내용이 없음", "confidence": 0.0}

    parser = JsonOutputParser(pydantic_object=ProfanityAnalysis)
    prompt_template_str = """
    당신은 한국어 텍스트에서 명백하고 직접적인 욕설의 사용 여부를 판단하는 정확한 언어 분석 전문가입니다.
    다른 종류의 비방, 비판, 풍자, 은어, 간접적인 표현이 아닌, **오직 직접적으로 사용된 욕설 단어 자체의 존재 유무**에만 집중해주세요.

    [판단 지침]
    1.  **직접성**: 문맥이나 의도가 아니라, 텍스트에 실제 욕설로 분류될 수 있는 단어가 직접적으로 쓰였는지 확인합니다. (예: "씨발", "개새끼", "병신" 등)
    2.  **명확성**: 누구나 욕설로 인지할 수 있는 명확한 단어에 초점을 맞춥니다. 이외의 경우에는 "욕설 사용"으로 분류하지 않습니다.
    3.  **제외 대상**:
        * 단순한 비판이나 부정적인 감정 표현 (예: "정말 실망스럽다", "이건 아니지")
        * 풍자, 비꼬기, 은유적 표현 (예: "누구는 참 편하게 사네" -> 숨은 뜻이 있더라도 직접 욕설이 아니면 제외)
        * 온라인 커뮤니티의 일반적인 은어나 줄임말 (그 자체로 직접적인 욕설이 아니라면 제외)
        * 인용문이나 제3자의 발언 속에 포함된 욕설은, 작성자 본인이 직접 사용한 것이 아니라면 신중히 판단 (여기서는 작성자 본인이 사용한 경우로 가정)

    [게시글 정보]
    제목: "{post_title}" (참고용)
    내용:
    ---
    {text_content}
    ---

    [출력 형식] - 반드시 다음 JSON 스키마를 따라주세요.
    {format_instructions}

    [분석 결과 설명 작성 가이드]
    * **욕설 사용**으로 판단될 경우:
        * `classification`: "욕설 사용"
        * `reason`: "직접적인 욕설 포함됨" (또는 어떤 욕설이 감지되었는지 간략히 명시 가능: 예: "'씨X' 등 직접적인 욕설 사용")
        * `confidence_score`: 판단에 대한 높은 신뢰도 (예: 0.90 ~ 1.0)
    * **일반글** (직접적인 욕설 없음)으로 판단될 경우:
        * `classification`: "일반글"
        * `reason`: "직접적인 욕설 없음" (또는 "해당 없음")
        * `confidence_score`: 판단에 대한 신뢰도 (욕설이 없다는 판단에 대한 확신 정도)
    """
    
    prompt = ChatPromptTemplate.from_template(
        template=prompt_template_str,
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )
    
    chain = prompt | llm | parser

    try:
        analysis_result = chain.invoke({
            "post_title": post_title,
            "text_content": text_content
        })
        
        # Pydantic 모델이 정상적으로 파싱되었는지 확인
        classification = analysis_result.classification if hasattr(analysis_result, 'classification') else "분석 실패 (파싱 오류)"
        reason = analysis_result.reason if hasattr(analysis_result, 'reason') else "이유 분석 실패"
        confidence = analysis_result.confidence_score if hasattr(analysis_result, 'confidence_score') else 0.0
            
        return {"classification": classification, "reason": reason, "confidence": confidence}

    except Exception as e:
        # print(f"LangChain 분석 중 오류 ({post_link}): {e}") # 상세 오류 확인용
        if "API key not valid" in str(e):
            print("오류: Gemini API 키가 유효하지 않습니다. .env 파일을 확인해주세요.")
        elif "429" in str(e) or "Resource has been exhausted" in str(e) or "quota" in str(e).lower():
            print(f"Gemini API 할당량 초과 가능성 ({post_link}). 잠시 후 수동 재시도 또는 스크립트 재실행 필요.")
            return {"classification": "분석 실패 (API 할당량)", "reason": str(e), "confidence": 0.0}
        return {"classification": "분석 실패", "reason": f"Gemini 응답 처리 오류: {str(e)[:100]}", "confidence": 0.0}

# --- 4. 메인 로직 ---
print(f"CSV 데이터 분석을 시작합니다 (LangChain, 모델: {LANGCHAIN_MODEL_NAME})...")

print(f"'{INPUT_CSV_FILE}'에서 데이터를 로드합니다.")
posts_to_analyze = load_data_from_csv(INPUT_CSV_FILE)

if not posts_to_analyze:
    print("로드된 데이터가 없습니다. 프로그램을 종료합니다.")
    exit() # exit() 대신 return을 사용하려면 이 코드가 main 함수 내에 있어야 합니다.

print(f"\n총 {len(posts_to_analyze)}개의 게시글 데이터 로드 완료.")
all_results = []

print("\n게시글 내용 분석 (LangChain Gemini API 사용) 중...")
for post_data in tqdm(posts_to_analyze, desc="게시글 분석 중"):
    post_title = post_data['title']
    post_link = post_data['link']
    post_content_excerpt = post_data['content'] # 'content_excerpt'를 'content'로 받아옴

    if not post_content_excerpt or not post_content_excerpt.strip():
        analysis = {"classification": "분석 불가 (내용 없음)", "reason": "content_excerpt가 비어있음", "confidence": 0.0}
    else:
        analysis_attempts = 0
        max_analysis_attempts = 3 # API 호출 재시도 횟수 (기존 2회에서 약간 늘림)
        analysis = None
        while analysis_attempts < max_analysis_attempts:
            analysis = analyze_text_with_langchain(post_content_excerpt, post_title, post_link)
            if analysis and analysis["classification"] != "분석 실패 (API 할당량)":
                break 
            analysis_attempts +=1
            if analysis_attempts < max_analysis_attempts and analysis and analysis["classification"] == "분석 실패 (API 할당량)":
                wait_time = 20 * (2 ** (analysis_attempts -1)) # 재시도 시 대기 시간 증가 (20초, 40초, 80초...)
                print(f"Gemini API 할당량 문제로 {wait_time}초 대기 후 재시도 ({analysis_attempts}/{max_analysis_attempts})... ({post_link})")
                time.sleep(wait_time)
            elif analysis_attempts < max_analysis_attempts :
                # 그 외 분석 실패 (네트워크 오류 등) 시 짧은 대기 후 재시도
                print(f"분석 실패로 5초 대기 후 재시도 ({analysis_attempts}/{max_analysis_attempts})... ({post_link})")
                time.sleep(5)
        
        if not analysis: # 모든 재시도 실패 시
             analysis = {"classification": "분석 최종 실패", "reason": "최대 재시도 도달", "confidence": 0.0}


    all_results.append({
        "title": post_title,
        "link": post_link,
        "content_excerpt": post_content_excerpt[:200].replace('\n', ' ') + "..." if post_content_excerpt else "내용 없음", # CSV 저장시에는 원본 content_excerpt 일부
        "gemini_classification": analysis.get("classification", "분석 실패"),
        "gemini_reason": analysis.get("reason", "N/A"),
        "gemini_confidence": f"{analysis.get('confidence', 0.0):.2f}"
    })
    # 각 게시물 처리 후 짧은 휴식 (API에 대한 부하 감소)
    time.sleep(random.uniform(1.0, 1.5)) # API 요청 간격을 약간 늘림 (기존 0.8~2.0에서 변경)

# --- 5. 결과를 CSV 파일로 저장 ---
print(f"\n분석 결과를 '{OUTPUT_CSV_FILE}' 파일로 저장합니다.")
profanity_posts_count = 0
if not all_results:
    print("저장할 분석 결과가 없습니다.")
else:
    try:
        # 디렉터리가 없는 경우 생성
        output_dir = os.path.dirname(OUTPUT_CSV_FILE)
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir)
            print(f"출력 디렉터리 '{output_dir}'를 생성했습니다.")

        with open(OUTPUT_CSV_FILE, mode='w', newline='', encoding='utf-8-sig') as file:
            writer = csv.DictWriter(file, fieldnames=["title", "link", "content_excerpt", "gemini_classification", "gemini_reason", "gemini_confidence"])
            writer.writeheader()
            for result in all_results:
                writer.writerow(result)
                if result["gemini_classification"] == "욕설 사용": # 변경된 분류명 확인
                    profanity_posts_count +=1
        print(f"'{OUTPUT_CSV_FILE}' 파일 저장 완료. 총 {len(all_results)}개 게시글 분석, {profanity_posts_count}개 '욕설 사용' 의심글 발견.")
    except IOError as e:
        print(f"오류: '{OUTPUT_CSV_FILE}' 파일 저장에 실패했습니다. - {e}")
    except Exception as e:
        print(f"오류: 파일 저장 중 알 수 없는 문제 발생 - {e}")


print("\n프로그램이 종료되었습니다.")

# 만약 이 스크립트를 직접 실행한다면 (예: python script_name.py)
# if __name__ == '__main__':
#   # 메인 로직을 함수로 감싸서 호출할 수 있습니다. 
#   # 예를 들어 main() 함수를 만들고 여기서 호출합니다.
#   # 현재는 스크립트 최상위 레벨에서 직접 실행됩니다.
#   pass

c:\Users\mmc\miniconda3\envs\filter\lib\site-packages\IPython\core\interactiveshell.py:3579: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


CSV 데이터 분석을 시작합니다 (LangChain, 모델: gemini-2.0-flash)...
'./data/raw_data.csv'에서 데이터를 로드합니다.
'./data/raw_data.csv'에서 총 300개의 데이터를 로드했습니다.

총 300개의 게시글 데이터 로드 완료.

게시글 내용 분석 (LangChain Gemini API 사용) 중...


게시글 분석 중:   6%|▌         | 17/300 [00:35<09:29,  2.01s/it]Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 41
}
].


Gemini API 할당량 초과 가능성 (https://gall.dcinside.com/board/view/?id=baseball_new11&no=18438493&page=1). 잠시 후 수동 재시도 또는 스크립트 재실행 필요.
Gemini API 할당량 문제로 20초 대기 후 재시도 (1/3)... (https://gall.dcinside.com/board/view/?id=baseball_new11&no=18438493&page=1)


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 19
}
].


...

Gemini API 할당량 초과 가능성 (https://gall.dcinside.com/board/view/?id=baseball_new11&no=18438233&page=6). 잠시 후 수동 재시도 또는 스크립트 재실행 필요.
Gemini API 할당량 문제로 20초 대기 후 재시도 (1/3)... (https://gall.dcinside.com/board/view/?id=baseball_new11&no=18438233&page=6)


게시글 분석 중:  93%|█████████▎| 278/300 [17:55<00:58,  2.66s/it]Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 21
}
].


Gemini API 할당량 초과 가능성 (https://gall.dcinside.com/board/view/?id=baseball_new11&no=18438215&page=6). 잠시 후 수동 재시도 또는 스크립트 재실행 필요.
Gemini API 할당량 문제로 20초 대기 후 재시도 (1/3)... (https://gall.dcinside.com/board/view/?id=baseball_new11&no=18438215&page=6)


게시글 분석 중:  98%|█████████▊| 293/300 [18:58<00:19,  2.73s/it]Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 18
}
].
게시글 분석 중: 100%|██████████| 300/300 [19:29<00:00,  3.90s/it]


분석 결과를 './data/1st_filtering.csv' 파일로 저장합니다.
'./data/1st_filtering.csv' 파일 저장 완료. 총 300개 게시글 분석, 0개 '욕설 사용' 의심글 발견.

프로그램이 종료되었습니다.


In [7]:
import os
import time
import csv
import re # 필요한 경우 텍스트 정제에 사용될 수 있으므로 유지합니다.
from dotenv import load_dotenv
import random # time.sleep()에 사용

# 웹 스크레이핑 관련 라이브러리는 더 이상 필요하지 않습니다.
# import requests
# from bs4 import BeautifulSoup
from tqdm import tqdm

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.pydantic_v1 import BaseModel, Field

# --- 1. 설정 ---
load_dotenv()
API_KEY = os.getenv('GEMINI_API') # .env 파일의 변수명을 GEMINI_API_KEY로 가정합니다 (일반적). 필요시 GEMINI_API로 수정하세요.

if not API_KEY:
    print("오류: GEMINI_API_KEY 환경 변수를 설정해주세요.")
    exit()

LANGCHAIN_MODEL_NAME = 'gemini-2.0-flash' # 사용 가능한 모델명으로 수정했습니다. 필요시 gemini-2.0-flash 등으로 변경하세요.

try:
    llm = ChatGoogleGenerativeAI(
        google_api_key=API_KEY,
        model=LANGCHAIN_MODEL_NAME,
        temperature=0.3, # 욕설 판단은 창의성보다 일관성이 중요하므로 온도를 낮춥니다.
        top_p=1,
        top_k=1,
    )
except Exception as e:
    print(f"LangChain LLM ({LANGCHAIN_MODEL_NAME}) 초기화 중 오류 발생: {e}")
    print("GEMINI_API_KEY가 정확하고, langchain_google_genai 라이브러리가 올바르게 설치되었는지, 모델명이 유효한지 확인해주세요.")
    exit()

INPUT_CSV_FILE = "./data/raw_data.csv" # 입력 CSV 파일 경로
OUTPUT_CSV_FILE = "./data/1st_filtering.csv" # 출력 CSV 파일명 변경
# HEADERS는 웹 스크레이핑에 사용되므로 더 이상 필요하지 않습니다.

# --- 2. CSV 데이터 로드 함수 ---
def load_data_from_csv(file_path):
    """CSV 파일에서 데이터를 읽어옵니다."""
    data = []
    try:
        with open(file_path, mode='r', encoding='utf-8-sig') as file:
            reader = csv.DictReader(file)
            if not reader.fieldnames or not all(col in reader.fieldnames for col in ['title', 'link', 'content_excerpt']):
                print(f"오류: CSV 파일 '{file_path}'에 'title', 'link', 'content_excerpt' 열이 모두 존재해야 합니다.")
                return []
            for row in reader:
                # 누락된 열이 있는지 확인하고, 있다면 기본값으로 채우거나 오류 처리
                title = row.get('title', '제목 없음')
                link = row.get('link', '링크 없음')
                content = row.get('content_excerpt', '') # 분석할 내용
                if not content: # content_excerpt가 비어있는 경우 분석에서 제외하거나 특별 처리 가능
                    print(f"경고: '{title}' 게시글의 'content_excerpt'가 비어있습니다. 분석에서 제외될 수 있습니다.")
                data.append({'title': title, 'link': link, 'content': content})
        print(f"'{file_path}'에서 총 {len(data)}개의 데이터를 로드했습니다.")
    except FileNotFoundError:
        print(f"오류: 입력 CSV 파일 '{file_path}'을(를) 찾을 수 없습니다.")
        return []
    except Exception as e:
        print(f"오류: CSV 파일 '{file_path}' 로드 중 문제 발생 - {e}")
        return []
    return data

# --- 3. LangChain을 사용한 텍스트 분석 함수 ---

# Pydantic 모델 정의 (LLM이 반환할 JSON 구조 명시)
class ProfanityAnalysis(BaseModel):
    classification: str = Field(description="게시글 내용에 직접적인 욕설이 포함되어 있으면 '욕설 사용', 아니면 '일반글'")
    reason: str = Field(description="분석 결과에 대한 이유. '욕설 사용' 시 '직접적인 욕설 포함됨'. '일반글' 시 '해당 없음' 또는 '직접적인 욕설 없음'")
    confidence_score: float = Field(description="분석 결과에 대한 신뢰도 점수 (0.0 ~ 1.0), 소수점 둘째자리까지")

def analyze_text_with_langchain(text_content, post_title, post_link):
    """LangChain과 ChatGoogleGenerativeAI를 사용하여 텍스트에 직접적인 욕설이 있는지 분석합니다."""
    if not text_content or not text_content.strip():
        return {"classification": "분석 불가", "reason": "분석할 내용이 없음", "confidence": 0.0}

    parser = JsonOutputParser(pydantic_object=ProfanityAnalysis)
    prompt_template_str = """
    당신은 한국어 텍스트에서 명백하고 직접적인 욕설의 사용 여부를 판단하는 정확한 언어 분석 전문가입니다.
    다른 종류의 비방, 비판, 풍자, 은어, 간접적인 표현이 아닌, **오직 직접적으로 사용된 욕설 단어 자체의 존재 유무**에만 집중해주세요.

    [판단 지침]
    1.  **직접성**: 문맥이나 의도가 아니라, 텍스트에 실제 욕설로 분류될 수 있는 단어가 직접적으로 쓰였는지 확인합니다. (예: "씨발", "개새끼", "병신" 등)
    2.  **명확성**: 누구나 욕설로 인지할 수 있는 명확한 단어에 초점을 맞춥니다. 이외의 경우에는 "욕설 사용"으로 분류하지 않습니다.
    3.  **제외 대상**:
        * 단순한 비판이나 부정적인 감정 표현 (예: "정말 실망스럽다", "이건 아니지")
        * 풍자, 비꼬기, 은유적 표현 (예: "누구는 참 편하게 사네" -> 숨은 뜻이 있더라도 직접 욕설이 아니면 제외)
        * 온라인 커뮤니티의 일반적인 은어나 줄임말 (그 자체로 직접적인 욕설이 아니라면 제외)
        * 인용문이나 제3자의 발언 속에 포함된 욕설은, 작성자 본인이 직접 사용한 것이 아니라면 신중히 판단 (여기서는 작성자 본인이 사용한 경우로 가정)

    [게시글 정보]
    제목: "{post_title}" (참고용)
    내용:
    ---
    {text_content}
    ---

    [출력 형식] - 반드시 다음 JSON 스키마를 따라주세요.
    {format_instructions}

    [분석 결과 설명 작성 가이드]
    * **욕설 사용**으로 판단될 경우:
        * `classification`: "욕설 사용"
        * `reason`: "직접적인 욕설 포함됨" (또는 어떤 욕설이 감지되었는지 간략히 명시 가능: 예: "'씨X' 등 직접적인 욕설 사용")
        * `confidence_score`: 판단에 대한 높은 신뢰도 (예: 0.90 ~ 1.0)
    * **일반글** (직접적인 욕설 없음)으로 판단될 경우:
        * `classification`: "일반글"
        * `reason`: "직접적인 욕설 없음" (또는 "해당 없음")
        * `confidence_score`: 판단에 대한 신뢰도 (욕설이 없다는 판단에 대한 확신 정도)
    """
    
    prompt = ChatPromptTemplate.from_template(
        template=prompt_template_str,
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )
    
    chain_upto_llm = prompt | llm 

    try:
        # LLM으로부터 원시 응답 받기
        raw_llm_response_message = chain_upto_llm.invoke({
            "post_title": post_title,
            "text_content": text_content
        })
        
        # AIMessage 객체의 content 속성에 실제 텍스트 응답이 들어있습니다.
        actual_response_text = raw_llm_response_message.content
        
        # print(f"DEBUG: Raw LLM Output for post '{post_title}':\n---\n{actual_response_text}\n---") # <--- 이 부분을 확인하세요!

        # 이제 이 텍스트를 파서로 파싱 시도
        # JsonOutputParser는 문자열 입력을 기대하므로 .content를 전달합니다.
        # analysis_result = parser.parse(actual_response_text) 
        
        # classification = analysis_result.classification
        # reason = analysis_result.reason
        # confidence = analysis_result.confidence_score
            
        # return {"classification": classification, "reason": reason, "confidence": confidence}
        parsed_dict = parser.parse(actual_response_text) 
        
        classification = parsed_dict.get("classification", "파싱 오류 (키 없음)") # dict 접근
        reason = parsed_dict.get("reason", "파싱 오류 (키 없음)")
        # confidence_score는 float으로 받아야 하므로, float 변환 시도
        confidence_str = parsed_dict.get("confidence_score", "0.0")
        try:
            confidence = float(confidence_str)
        except ValueError:
            print(f"경고: confidence_score를 float으로 변환 실패. 값: {confidence_str}")
            confidence = 0.0
            
        return {"classification": classification, "reason": reason, "confidence": confidence}

    except Exception as e:
        print(f"오류 발생 지점: LangChain 분석 중 ({post_link})")
        print(f"오류 메시지: {e}")
        # 오류 발생 시에도 실제 LLM 응답이 있었다면 출력
        if 'actual_response_text' in locals() and actual_response_text:
             print(f"DEBUG: Raw LLM Output that caused parsing error for post '{post_title}':\n---\n{actual_response_text}\n---")

    except Exception as e:
        # print(f"LangChain 분석 중 오류 ({post_link}): {e}") # 상세 오류 확인용
        if "API key not valid" in str(e):
            print("오류: Gemini API 키가 유효하지 않습니다. .env 파일을 확인해주세요.")
        elif "429" in str(e) or "Resource has been exhausted" in str(e) or "quota" in str(e).lower():
            print(f"Gemini API 할당량 초과 가능성 ({post_link}). 잠시 후 수동 재시도 또는 스크립트 재실행 필요.")
            return {"classification": "분석 실패 (API 할당량)", "reason": str(e), "confidence": 0.0}
        return {"classification": "분석 실패", "reason": f"Gemini 응답 처리 오류: {str(e)[:100]}", "confidence": 0.0}

# --- 4. 메인 로직 ---
print(f"CSV 데이터 분석을 시작합니다 (LangChain, 모델: {LANGCHAIN_MODEL_NAME})...")

print(f"'{INPUT_CSV_FILE}'에서 데이터를 로드합니다.")
posts_to_analyze = load_data_from_csv(INPUT_CSV_FILE)

if not posts_to_analyze:
    print("로드된 데이터가 없습니다. 프로그램을 종료합니다.")
    exit() # exit() 대신 return을 사용하려면 이 코드가 main 함수 내에 있어야 합니다.

print(f"\n총 {len(posts_to_analyze)}개의 게시글 데이터 로드 완료.")
all_results = []

print("\n게시글 내용 분석 (LangChain Gemini API 사용) 중...")
for post_data in tqdm(posts_to_analyze, desc="게시글 분석 중"):
    post_title = post_data['title']
    post_link = post_data['link']
    post_content_excerpt = post_data['content'] # 'content_excerpt'를 'content'로 받아옴

    if not post_content_excerpt or not post_content_excerpt.strip():
        analysis = {"classification": "분석 불가 (내용 없음)", "reason": "content_excerpt가 비어있음", "confidence": 0.0}
    else:
        analysis_attempts = 0
        max_analysis_attempts = 3 # API 호출 재시도 횟수 (기존 2회에서 약간 늘림)
        analysis = None
        while analysis_attempts < max_analysis_attempts:
            analysis = analyze_text_with_langchain(post_content_excerpt, post_title, post_link)
            if analysis and analysis["classification"] != "분석 실패 (API 할당량)":
                break 
            analysis_attempts +=1
            if analysis_attempts < max_analysis_attempts and analysis and analysis["classification"] == "분석 실패 (API 할당량)":
                wait_time = 20 * (2 ** (analysis_attempts -1)) # 재시도 시 대기 시간 증가 (20초, 40초, 80초...)
                print(f"Gemini API 할당량 문제로 {wait_time}초 대기 후 재시도 ({analysis_attempts}/{max_analysis_attempts})... ({post_link})")
                time.sleep(wait_time)
            elif analysis_attempts < max_analysis_attempts :
                # 그 외 분석 실패 (네트워크 오류 등) 시 짧은 대기 후 재시도
                print(f"분석 실패로 5초 대기 후 재시도 ({analysis_attempts}/{max_analysis_attempts})... ({post_link})")
                time.sleep(5)
        
        if not analysis: # 모든 재시도 실패 시
             analysis = {"classification": "분석 최종 실패", "reason": "최대 재시도 도달", "confidence": 0.0}


    all_results.append({
        "title": post_title,
        "link": post_link,
        "content_excerpt": post_content_excerpt[:200].replace('\n', ' ') + "..." if post_content_excerpt else "내용 없음", # CSV 저장시에는 원본 content_excerpt 일부
        "gemini_classification": analysis.get("classification", "분석 실패"),
        "gemini_reason": analysis.get("reason", "N/A"),
        "gemini_confidence": f"{analysis.get('confidence', 0.0):.2f}"
    })
    # 각 게시물 처리 후 짧은 휴식 (API에 대한 부하 감소)
    time.sleep(random.uniform(1.0, 1.5)) # API 요청 간격을 약간 늘림 (기존 0.8~2.0에서 변경)

# --- 5. 결과를 CSV 파일로 저장 ---
print(f"\n분석 결과를 '{OUTPUT_CSV_FILE}' 파일로 저장합니다.")
profanity_posts_count = 0
if not all_results:
    print("저장할 분석 결과가 없습니다.")
else:
    try:
        # 디렉터리가 없는 경우 생성
        output_dir = os.path.dirname(OUTPUT_CSV_FILE)
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir)
            print(f"출력 디렉터리 '{output_dir}'를 생성했습니다.")

        with open(OUTPUT_CSV_FILE, mode='w', newline='', encoding='utf-8-sig') as file:
            writer = csv.DictWriter(file, fieldnames=["title", "link", "content_excerpt", "gemini_classification", "gemini_reason", "gemini_confidence"])
            writer.writeheader()
            for result in all_results:
                writer.writerow(result)
                if result["gemini_classification"] == "욕설 사용": # 변경된 분류명 확인
                    profanity_posts_count +=1
        print(f"'{OUTPUT_CSV_FILE}' 파일 저장 완료. 총 {len(all_results)}개 게시글 분석, {profanity_posts_count}개 '욕설 사용' 의심글 발견.")
    except IOError as e:
        print(f"오류: '{OUTPUT_CSV_FILE}' 파일 저장에 실패했습니다. - {e}")
    except Exception as e:
        print(f"오류: 파일 저장 중 알 수 없는 문제 발생 - {e}")


print("\n프로그램이 종료되었습니다.")

# 만약 이 스크립트를 직접 실행한다면 (예: python script_name.py)
# if __name__ == '__main__':
#   # 메인 로직을 함수로 감싸서 호출할 수 있습니다. 
#   # 예를 들어 main() 함수를 만들고 여기서 호출합니다.
#   # 현재는 스크립트 최상위 레벨에서 직접 실행됩니다.
#   pass

CSV 데이터 분석을 시작합니다 (LangChain, 모델: gemini-2.0-flash)...
'./data/raw_data.csv'에서 데이터를 로드합니다.
'./data/raw_data.csv'에서 총 300개의 데이터를 로드했습니다.

총 300개의 게시글 데이터 로드 완료.

게시글 내용 분석 (LangChain Gemini API 사용) 중...


게시글 분석 중:   5%|▌         | 16/300 [00:33<10:58,  2.32s/it]Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 35
}
].
게시글 분석 중:   6%|▌         | 17/300 [00:38<13:28,  2.86s/it]Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in

오류 발생 지점: LangChain 분석 중 (https://gall.dcinside.com/board/view/?id=baseball_new11&no=18438493&page=1)
오류 메시지: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 28
}
]
분석 실패로 5초 대기 후 재시도 (1/3)... (https://gall.dcinside.com/board/view/?id=baseball_new11&no=18438493&page=1)


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 23
}
].


오류 발생 지점: LangChain 분석 중 (https://gall.dcinside.com/board/view/?id=baseball_new11&no=18438493&page=1)
오류 메시지: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 21
}
]
분석 실패로 5초 대기 후 재시도 (2/3)... (https://gall.dcinside.com/board/view/?id=baseball_new11&no=18438493&page=1)


...

오류 발생 지점: LangChain 분석 중 (https://gall.dcinside.com/board/view/?id=baseball_new11&no=18438193&page=6)
오류 메시지: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 20
}
]
분석 실패로 5초 대기 후 재시도 (2/3)... (https://gall.dcinside.com/board/view/?id=baseball_new11&no=18438193&page=6)


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 15
}
].


오류 발생 지점: LangChain 분석 중 (https://gall.dcinside.com/board/view/?id=baseball_new11&no=18438193&page=6)
오류 메시지: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 13
}
]


게시글 분석 중: 100%|█████████▉| 299/300 [16:59<00:05,  5.52s/it]Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 9
}
].
게시글 분석 중: 100%|██████████| 300/300 [17:03<00:00,  3.41s/it]


분석 결과를 './data/1st_filtering.csv' 파일로 저장합니다.
오류: './data/1st_filtering.csv' 파일 저장에 실패했습니다. - [Errno 13] Permission denied: './data/1st_filtering.csv'

프로그램이 종료되었습니다.


In [8]:
print(f"\n분석 결과를 '{OUTPUT_CSV_FILE}' 파일로 저장합니다.")
profanity_posts_count = 0
if not all_results:
    print("저장할 분석 결과가 없습니다.")
else:
    try:
        # 디렉터리가 없는 경우 생성
        output_dir = os.path.dirname(OUTPUT_CSV_FILE)
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir)
            print(f"출력 디렉터리 '{output_dir}'를 생성했습니다.")

        with open(OUTPUT_CSV_FILE, mode='w', newline='', encoding='utf-8-sig') as file:
            writer = csv.DictWriter(file, fieldnames=["title", "link", "content_excerpt", "gemini_classification", "gemini_reason", "gemini_confidence"])
            writer.writeheader()
            for result in all_results:
                writer.writerow(result)
                if result["gemini_classification"] == "욕설 사용": # 변경된 분류명 확인
                    profanity_posts_count +=1
        print(f"'{OUTPUT_CSV_FILE}' 파일 저장 완료. 총 {len(all_results)}개 게시글 분석, {profanity_posts_count}개 '욕설 사용' 의심글 발견.")
    except IOError as e:
        print(f"오류: '{OUTPUT_CSV_FILE}' 파일 저장에 실패했습니다. - {e}")
    except Exception as e:
        print(f"오류: 파일 저장 중 알 수 없는 문제 발생 - {e}")


분석 결과를 './data/1st_filtering.csv' 파일로 저장합니다.
'./data/1st_filtering.csv' 파일 저장 완료. 총 300개 게시글 분석, 133개 '욕설 사용' 의심글 발견.
